In [1]:
import sys

sys.path.append('./source/')

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import integrate, stats
import matplotlib.cm as cm
from numpy import fft
from scipy import interpolate as intp
import importlib
import healpy

from source import plots
from source import prob_dists as pd
from utils import read_param_file, update_params, configure_plot_params

configure_plot_params(fontsize=12)

In [3]:
param_file = './source/n0.params'
params = read_param_file(param_file)
p = params

# exposure = 5 * 14000 / (4 * np.pi)
# p = update_params(param_file, p, 'exposure', exposure)

# phipp = 7.12073e-30
# fwimp = phipp * 8 * np.pi / 1e-28
# # fwimp = 1e-4
# p = update_params(param_file, p, 'fwimp', fwimp)

# p = update_params(param_file, p, 'bg', 1)
# p = update_params(param_file, p, 'beg', 1)

# print(exposure, fwimp)
p

{'R_G': 220,
 'd_solar': 8.5,
 'psi': 40,
 'M_min': 0.1,
 'M_max': 10000000000.0,
 'fwimp': 1.78,
 'mean_params': {'a': 77.4, 'b': 0.87, 'c': -0.23},
 'nside': 128,
 'area_fermi': 2000.0,
 'n': 0,
 'log_flux_min': -16,
 'log_flux_max': -2,
 'N': 2200,
 'log_k_min': -2,
 'log_k_max': 10,
 'N_k': 250000,
 'psh_log_f_min': -7,
 'psh_log_f_max': -1.0,
 'N_psh': 1000,
 'omega_pixel': 6.391586616190171e-05,
 'exposure': 5570.423008216337,
 'iso_flux_bg': 0.0008615905978150363,
 'gal_flux_bg_file': './output/gal_flux_bg.npy',
 'bg': 1,
 'beg': 1,
 'beta': 1.9}

In [4]:
nside = p['nside']
npix = healpy.nside2npix(nside)
lon, lat = healpy.pix2ang(nside, range(npix), lonlat=True)
ang_dists = np.rad2deg(np.arccos(np.cos(np.deg2rad(lon)) * np.cos(np.deg2rad(lat))))
good_indices = (abs(lat) >= 40)

In [5]:
## S-WAVE
s_psh, s_pshfunc2d, s_fluxes, _ = pd.psh_s(ang_dists[good_indices], input_file='./output/n0_pshfunc_paper.npz', return_all=True)
s_psh = s_psh[::1]
s_fluxes = s_fluxes[::1]
print(s_psh.shape)
print(np.trapz(s_psh, s_fluxes, axis=0))
s_psh /= np.trapz(s_psh, s_fluxes, axis=0)
print(np.trapz(s_psh, s_fluxes, axis=0))

## SOM
som_fwimp_rescale = 10.6404808663
# som_fwimp_rescale = 13.333232303696379
som_psh, som_pshfunc2d, som_fluxes, _= pd.psh_som(ang_dists[good_indices], input_file='./output/n-1_pshfunc_paper.npz', return_all=True, rescale=som_fwimp_rescale)
som_psh = som_psh[::1]
som_fluxes = som_fluxes[::1]
print(np.trapz(som_psh, som_fluxes, axis=0))
som_psh /= np.trapz(som_psh, som_fluxes, axis=0)
print(som_psh.shape, som_fluxes.shape)

# som_psh /= som_fwimp_rescale
# som_fluxes *= som_fwimp_rescale
print(np.trapz(som_psh, som_fluxes, axis=0))

# print(s_fluxes, som_fluxes)

(100, 70144)
[1.0005111  1.0005111  1.0005111  ... 1.00051914 1.00051914 1.00051914]
[1. 1. 1. ... 1. 1. 1.]
[1.00028714 1.00028714 1.00028714 ... 1.00028698 1.00028698 1.00028698]
(100, 70144) (100,)
[1. 1. 1. ... 1. 1. 1.]


In [6]:
# search parameters
exposure_search = [p['exposure']]
begs = []
fwimps = []

In [7]:
def likelihood_run_for_model_2d(p, psh, fluxes, poisson_rescale=1, model='', som_fwimp_rescale=1, s_fwimp_rescale=1, bgd_mismodel=1, bgd_undermodel=1, minimize=False, load_data_file=None):
    # run likelihoods
    for i, exposure in enumerate(exposure_search):
        p['exposure'] = exposure

        gal_bg = np.load(p['gal_flux_bg_file'])[good_indices] * p['exposure'] * p['bg']
        iso_bg = p['iso_flux_bg'] * p['exposure'] * p['beg']
        bg_count = gal_bg * bgd_mismodel + iso_bg

        counts = np.arange(0, bg_count.max() + 3.5 * np.sqrt(bg_count.max()) + p['exposure'] * p['fwimp'] * fluxes.max())

        print('calculating pc to count =', counts[-1])

        unique_ang, uni_ind, uni_inv = np.unique(np.abs(ang_dists[good_indices]), return_inverse=True, return_index=True)
        
        print('psh mean', np.trapz(fluxes * psh[:, 0], fluxes))

        pshdat = psh[:, uni_ind]
        psh = pshdat[:, uni_inv]
        if model == 's':
            spsh = pshdat[:, uni_inv]
            sompsh = som_psh
        elif model == 'som':
            sompsh = pshdat[:, uni_inv]
            spsh = s_psh
        else:
            spsh = s_psh
            sompsh = som_psh

        if load_data_file is None:
            if model == '':
                pc_psi = stats.poisson.pmf(counts[np.newaxis, :], np.trapz(psh * fluxes[:, np.newaxis] * p['exposure'] * p['fwimp'], fluxes[:, np.newaxis], axis=0)[:, np.newaxis])
            else:
                pc_psi = integrate.simps(pshdat[..., np.newaxis] * stats.poisson.pmf(counts[np.newaxis, np.newaxis, :], p['exposure'] * p['fwimp'] * fluxes[:, np.newaxis, np.newaxis]), fluxes, axis=0)
                pc_psi /= np.sum(pc_psi, axis=-1)[:, np.newaxis]

            print('is pc norm', np.allclose(np.sum(pc_psi, axis=-1), 1))
            print('last pc prob', pc_psi[:, -1])

            pc_of_psi = pc_psi[uni_inv]

            # generate sky map
            subcounts = pd.generate_skymap_sample_pc(p, pc_of_psi, ang_dists[good_indices], good_indices, return_subcounts=True, save_output=True, bg_counts=bg_count)

            print('generated skymap with', p['fwimp'])
            print('max counts', counts[-1], subcounts.max())
        else:
            subcounts = np.load(load_data_file)[good_indices]
            print('loaded skymap from', load_data_file)
            print('max counts', counts[-1], subcounts.max())

        if minimize is True:
            from scipy.optimize import minimize

#             indmax = np.unravel_index(np.argmin(s_S, axis=None), s_S.shape)

            s_max = minimize(pd.likelihood2d, [exposure_search[0], p['beg'], p['fwimp']], args=(spsh, subcounts.astype(np.int16), s_fluxes, iso_bg, gal_bg*bgd_undermodel), method='Nelder-Mead')
#             print('swave\t', s_max)

            som_max = minimize(pd.likelihood2d, [exposure_search[0], p['beg'], p['fwimp']], args=(sompsh, subcounts.astype(np.int16), som_fluxes, iso_bg, gal_bg*bgd_undermodel), method='Nelder-Mead')
#             print('som\t', som_max)

            poi_max = minimize(pd.poisson_likelihood2d, [exposure_search[0], p['beg'], p['fwimp']], args=(psh, subcounts.astype(np.int16), fluxes, iso_bg, gal_bg*bgd_undermodel), method='Nelder-Mead')
#             print('poi\t', poi_max)
            return [s_max.fun, som_max.fun, poi_max.fun] 

    return None

In [8]:
s_file = './output/nNull_skymap_48061.npy'
som_file = './output/n-1_skymap_90168.npy'
poi_file = './output/nNull_skymap_36658.npy'
# s_file = None
# som_file = None
# poi_file = None

In [9]:
SS = [[],[],[]]

p['n'] = 0
SS[0] = likelihood_run_for_model_2d(p, s_psh, s_fluxes, model='s', minimize=True, load_data_file=s_file)
print('for s-wave generated data')
print('delta S som-s:', SS[0][1] - SS[0][0])
print('delta S poi-s:', SS[0][2] - SS[0][0])

p['n'] = -1
SS[1] = likelihood_run_for_model_2d(p, som_psh, som_fluxes, model='som', minimize=True, load_data_file=som_file)
print('for som-wave generated data')
print('delta S s-som:', SS[1][0] - SS[1][1])
print('delta S poi-som:', SS[1][2] - SS[1][1])

p['n'] = 'Null'
SS[2] = likelihood_run_for_model_2d(p, som_psh, som_fluxes, model='', minimize=True, load_data_file=poi_file)
print('for poisson generated data')
print('delta S s-poi:', SS[2][0] - SS[2][2])
print('delta S som-poi:', SS[2][1] - SS[2][2])

print('\n for s-wave generated data')
print('delta S som-s:', SS[0][1] - SS[0][0])
print('delta S poi-s:', SS[0][2] - SS[0][0])
print('\n for som-wave generated data')
print('delta S s-som:', SS[1][0] - SS[1][1])
print('delta S poi-som:', SS[1][2] - SS[1][1])
print('\n for poisson generated data')
print('delta S s-poi:', SS[2][0] - SS[2][2])
print('delta S som-poi:', SS[2][1] - SS[2][2])


calculating pc to count = 94.0
psh mean 6.847131703151762e-05
loaded skymap from ./output/nNull_skymap_48061.npy
max counts 94.0 56.0
for s-wave generated data
delta S som-s: 71.14946259965654
delta S poi-s: 42.64234784204746
calculating pc to count = 91.0
psh mean 7.384715547535348e-05
loaded skymap from ./output/n-1_skymap_90168.npy
max counts 91.0 63.0
for som-wave generated data
delta S s-som: 92.63479357241886
delta S poi-som: 49.41937920911005
calculating pc to count = 91.0
psh mean 7.384715547535348e-05
loaded skymap from ./output/nNull_skymap_36658.npy
max counts 91.0 60.0
for poisson generated data
delta S s-poi: 71.88326974149095
delta S som-poi: 39.81708630558569

 for s-wave generated data
delta S som-s: 71.14946259965654
delta S poi-s: 42.64234784204746

 for som-wave generated data
delta S s-som: 92.63479357241886
delta S poi-som: 49.41937920911005

 for poisson generated data
delta S s-poi: 71.88326974149095
delta S som-poi: 39.81708630558569


In [10]:
SS_correct = SS
print(SS)

[[371548.8593609691, 371620.00882356876, 371591.50170881115], [369891.812264947, 369799.17747137457, 369848.5968505837], [371045.91741290287, 371013.85122946696, 370974.0341431614]]


In [11]:
SS = [[],[],[]]

p['n'] = 0
SS[0] = likelihood_run_for_model_2d(p, s_psh, s_fluxes, model='s', minimize=True, bgd_undermodel=1.03, load_data_file=s_file)
print('for s-wave generated data')
print('delta S som-s:', SS[0][1] - SS[0][0])
print('delta S poi-s:', SS[0][2] - SS[0][0])

p['n'] = -1
SS[1] = likelihood_run_for_model_2d(p, som_psh, som_fluxes, model='som', minimize=True, bgd_undermodel=1.03, load_data_file=som_file)
print('for som-wave generated data')
print('delta S s-som:', SS[1][0] - SS[1][1])
print('delta S poi-som:', SS[1][2] - SS[1][1])

p['n'] = 'Null'
SS[2] = likelihood_run_for_model_2d(p, som_psh, som_fluxes, model='', minimize=True, bgd_undermodel=1.03, load_data_file=poi_file)
print('for poisson generated data')
print('delta S s-poi:', SS[2][0] - SS[2][2])
print('delta S som-poi:', SS[2][1] - SS[2][2])

print('\n for s-wave generated data')
print('delta S som-s:', SS[0][1] - SS[0][0])
print('delta S poi-s:', SS[0][2] - SS[0][0])
print('\n for som-wave generated data')
print('delta S s-som:', SS[1][0] - SS[1][1])
print('delta S poi-som:', SS[1][2] - SS[1][1])
print('\n for poisson generated data')
print('delta S s-poi:', SS[2][0] - SS[2][2])
print('delta S som-poi:', SS[2][1] - SS[2][2])

SS_overmodel = SS

calculating pc to count = 94.0
psh mean 6.847131703151762e-05
loaded skymap from ./output/nNull_skymap_48061.npy
max counts 94.0 56.0
for s-wave generated data
delta S som-s: 110.63243094604695
delta S poi-s: 87.1426001614891
calculating pc to count = 91.0
psh mean 7.384715547535348e-05
loaded skymap from ./output/n-1_skymap_90168.npy
max counts 91.0 63.0
for som-wave generated data
delta S s-som: 134.86380296153948
delta S poi-som: 98.80530873284442
calculating pc to count = 91.0
psh mean 7.384715547535348e-05
loaded skymap from ./output/nNull_skymap_36658.npy
max counts 91.0 60.0
for poisson generated data
delta S s-poi: 70.52817848121049
delta S som-poi: 38.46774887916399

 for s-wave generated data
delta S som-s: 110.63243094604695
delta S poi-s: 87.1426001614891

 for som-wave generated data
delta S s-som: 134.86380296153948
delta S poi-som: 98.80530873284442

 for poisson generated data
delta S s-poi: 70.52817848121049
delta S som-poi: 38.46774887916399


In [12]:
SS = [[],[],[]]

p['n'] = 0
SS[0] = likelihood_run_for_model_2d(p, s_psh, s_fluxes, model='s', minimize=True, bgd_undermodel=0.97, load_data_file=s_file)
print('for s-wave generated data')
print('delta S som-s:', SS[0][1] - SS[0][0])
print('delta S poi-s:', SS[0][2] - SS[0][0])

p['n'] = -1
SS[1] = likelihood_run_for_model_2d(p, som_psh, som_fluxes, model='som', minimize=True, bgd_undermodel=0.97, load_data_file=som_file)
print('for som-wave generated data')
print('delta S s-som:', SS[1][0] - SS[1][1])
print('delta S poi-som:', SS[1][2] - SS[1][1])

p['n'] = 'Null'
SS[2] = likelihood_run_for_model_2d(p, som_psh, som_fluxes, model='', minimize=True, bgd_undermodel=0.97, load_data_file=poi_file)
print('for poisson generated data')
print('delta S s-poi:', SS[2][0] - SS[2][2])
print('delta S som-poi:', SS[2][1] - SS[2][2])

print('\n for s-wave generated data')
print('delta S som-s:', SS[0][1] - SS[0][0])
print('delta S poi-s:', SS[0][2] - SS[0][0])
print('\n for som-wave generated data')
print('delta S s-som:', SS[1][0] - SS[1][1])
print('delta S poi-som:', SS[1][2] - SS[1][1])
print('\n for poisson generated data')
print('delta S s-poi:', SS[2][0] - SS[2][2])
print('delta S som-poi:', SS[2][1] - SS[2][2])

SS_undermodel = SS

calculating pc to count = 94.0
psh mean 6.847131703151762e-05
loaded skymap from ./output/nNull_skymap_48061.npy
max counts 94.0 56.0
for s-wave generated data
delta S som-s: 35.185261879872996
delta S poi-s: 2.5275105320615694
calculating pc to count = 91.0
psh mean 7.384715547535348e-05
loaded skymap from ./output/n-1_skymap_90168.npy
max counts 91.0 63.0
for som-wave generated data
delta S s-som: 59.76093348697759
delta S poi-som: 10.330233009299263
calculating pc to count = 91.0
psh mean 7.384715547535348e-05
loaded skymap from ./output/nNull_skymap_36658.npy
max counts 91.0 60.0
for poisson generated data
delta S s-poi: 72.33549139439128
delta S som-poi: 40.28287842165446

 for s-wave generated data
delta S som-s: 35.185261879872996
delta S poi-s: 2.5275105320615694

 for som-wave generated data
delta S s-som: 59.76093348697759
delta S poi-som: 10.330233009299263

 for poisson generated data
delta S s-poi: 72.33549139439128
delta S som-poi: 40.28287842165446


In [13]:
SS

[[371590.9213507674, 371626.1066126473, 371593.4488612995],
 [369929.98433583864, 369870.22340235166, 369880.55363536096],
 [371149.6112093341, 371117.55859636137, 371077.2757179397]]

In [14]:
np.array(SS_correct) - np.array(SS)

array([[ -42.0619898 ,   -6.09778908,   -1.94715249],
       [ -38.17207089,  -71.04593098,  -31.95678478],
       [-103.69379643, -103.70736689, -103.24157478]])

In [19]:
np.diag( - np.array(SS_correct) + np.array(SS_undermodel))/2

array([21.0309949 , 35.52296549, 51.62078739])

In [20]:

np.diag( - np.array(SS_correct) + np.array(SS_overmodel))/2

array([30.92043777, 13.46859619,  2.43739016])